In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

from IPython.display import Markdown, display
def hr(): display(Markdown('---'))
def cprint(msg: str, color: str = "blue", **kwargs) -> str:
    if color == "blue": print("\033[34m" + msg + "\033[0m", **kwargs)
    elif color == "red": print("\033[31m" + msg + "\033[0m", **kwargs)
    elif color == "green": print("\033[32m" + msg + "\033[0m", **kwargs)
    elif color == "yellow": print("\033[33m" + msg + "\033[0m", **kwargs)
    elif color == "purple": print("\033[35m" + msg + "\033[0m", **kwargs)
    elif color == "cyan": print("\033[36m" + msg + "\033[0m", **kwargs)
    else: raise ValueError(f"Invalid info color: `{color}`")

In [2]:
model_name = "stabilityai/stablelm-tuned-alpha-3b" #@param ["stabilityai/stablelm-tuned-alpha-7b", "stabilityai/stablelm-base-alpha-7b", "stabilityai/stablelm-tuned-alpha-3b", "stabilityai/stablelm-base-alpha-3b"]

cprint(f"Using `{model_name}`", color="blue")

# Select "big model inference" parameters
torch_dtype = "float16" #@param ["float16", "bfloat16", "float"]
load_in_8bit = False #@param {type:"boolean"}
device_map = "auto"

cprint(f"Loading with: `{torch_dtype=}, {load_in_8bit=}, {device_map=}`")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=getattr(torch, torch_dtype),
    load_in_8bit=load_in_8bit,
    device_map=device_map,
    offload_folder="./offload",
)

Using `stabilityai/stablelm-tuned-alpha-3b`
Loading with: `torch_dtype='float16', load_in_8bit=False, device_map='auto'`


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aviv9\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

# Process the user prompt
user_prompt = "Can you write a song about a pirate at sea?" #@param {type:"string"}
if "tuned" in model_name:
  # Add system prompt for chat tuned models
  system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
  - StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
  - StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
  - StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
  - StableLM will refuse to participate in anything that could harm a human.
  """
  prompt = f"{system_prompt}<|USER|>{user_prompt}<|ASSISTANT|>"
else:
  prompt = user_prompt

# Sampling args
max_new_tokens = 128 #@param {type:"slider", min:32.0, max:3072.0, step:32}
temperature = 0.7 #@param {type:"slider", min:0.0, max:1.25, step:0.05}
top_k = 0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
do_sample = True #@param {type:"boolean"}

cprint(f"Sampling with: `{max_new_tokens=}, {temperature=}, {top_k=}, {top_p=}, {do_sample=}`")
hr()

# Create `generate` inputs
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(model.device)

# Generate
tokens = model.generate(
  **inputs,
  max_new_tokens=max_new_tokens,
  temperature=temperature,
  top_k=top_k,
  top_p=top_p,
  do_sample=do_sample,
  pad_token_id=tokenizer.eos_token_id,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()])
)

# Extract out only the completion tokens
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

# Display
print(user_prompt + " ", end="")
cprint(completion, color="green")

Sampling with: `max_new_tokens=128, temperature=0.7, top_k=0, top_p=0.9, do_sample=True`


---

Can you write a song about a pirate at sea? Sure, here’s a song about a pirate who sailed the seven seas:

“We’re sailing in a great big sea,
With the wind at our backs and the waves a-plenty,
We’re searching for a rumored treasure,
A chest of gold and jewels beyond measure.

We’ve battled storms and faced the pirates,
And we’ve always come out victorious,
We’ve sailed the seven seas,
And we’ll sail them again and again.

We’ve found the treasure we sought,
But it was far beyond our reach


In [4]:
# try with pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, pipeline


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False


model_name = "stabilityai/stablelm-tuned-alpha-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gpu_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
    # offload_folder="./offload",
)

max_new_tokens = 1024
temperature = 0.1
top_k = 0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
do_sample = True #@param {type:"boolean"}


pipe = pipeline(
    "text-generation",
    model=gpu_model,
    tokenizer=tokenizer, 
    max_length=max_new_tokens,
    temperature=temperature,
    top_k = top_k,
    top_p = top_p,
    do_sample=do_sample,
    pad_token_id=tokenizer.eos_token_id,
    stopping_criteria=StoppingCriteriaList([StopOnTokens()])
)


CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a): Find the cuda library via: find / -name libcuda.so 2>/dev/null
CUDA SETUP: Solution 2b): Once the library is found add it to the LD_LIBRARY_PATH: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:FOUND_PATH_FROM_2a
CUDA SETUP: Solution 2c): For a permanent solution add the export from 2b in

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

llm = HuggingFacePipeline(pipeline=pipe)

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
  - StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
  - StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
  - StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
  - StableLM will refuse to participate in anything that could harm a human.
  """

prompt = PromptTemplate(
    input_variables=["user_prompt"], 
    template=system_prompt + "<|USER|>{user_prompt}<|ASSISTANT|>"
)

llm_chain = LLMChain(prompt=prompt, llm=llm, )

In [ ]:
print(llm_chain.predict(user_prompt="Write a kql function to calculate the std of a numerical column"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Here is a possible implementation of a kql function in Python:
```python
import math

def kql_to_std(col_name, col_type):
    """
    Returns the standard deviation of a numerical column.

    Args:
    col_name (str): The name of the column to calculate the standard deviation.
    col_type (str): The type of the column (e.g. int, float, str).

    Returns:
    float: The sum of all the values in the column.
    """
    if col_name == "std":
        return math.sqrt(col_type)
    elif col_type == "float":
        return float(col_name)
    elif col_type == "str":
        return str(col_name)
    else:
        return math.sqrt(col_type)
```
To use this function, you can call it with the name of the column and the desired type of the column (e.g. "std" for std column, "float" for float column, etc.). The function will return the sum of all the values in the column.

For example:
```python
col_name = "std"
col_type = "float"

std_sum = kql_to_std(col_name, col_type)
print(f"The standard d